# BERT-language2label

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Batcher ModelSupport Datasets TextModels

import Datasets
import Foundation
import ModelSupport
import TensorFlow
import TextModels
import PythonKit

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Batcher
		ModelSupport
		Datasets
		TextModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpym3mhhjj/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
Initializing Swift...
Installation complete!


In [2]:
let bertPretrained = BERT.PreTrainedModel.bertBase(cased: false, multilingual: false)
let workspaceURL = URL(
    fileURLWithPath: "bert_models", isDirectory: true,
    relativeTo: URL(
        fileURLWithPath: NSTemporaryDirectory(),
        isDirectory: true))
let bert = try BERT.PreTrainedModel.load(bertPretrained)(from: workspaceURL)
var bertClassifier = BERTClassifier(bert: bert, classCount: 5)

Loading BERT pre-trained model 'BERT Base Uncased'.
Loading resource: uncased_L-12_H-768_A-12


## load dataset

In [3]:
// TODO: 
// + configure dataset path outside of Language2Label
// + train

In [4]:
%include "/notebooks/language2motion.gt/code/Sources/BERT-language2label/Language2Label.swift"

In [5]:
let maxSequenceLength = 50
let batchSize = 2048

let dsURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/labels_ds_v2_manual_labels.csv")

var dataset = try Language2Label(
  datasetURL: dsURL,
  maxSequenceLength: maxSequenceLength,
  batchSize: batchSize,
  entropy: SystemRandomNumberGenerator()
) { (example: Language2LabelExample) -> LabeledTextBatch in
  let textBatch = bertClassifier.bert.preprocess(
    sequences: [example.text],
    maxSequenceLength: maxSequenceLength)
   return (data: textBatch, 
           label: example.label.map { 
               (label: Language2LabelExample.LabelTuple) in Tensor(Int32(label.idx))
           }!
          )
}

print("Dataset acquired.")

Dataset acquired.


In [6]:
dataset.trainingExamples.count

2409


In [7]:
dataset.trainingExamples[0]

▿ 2 elements
  ▿ data : TextBatch
    - tokenIds : [[ 101, 1037, 2711, 7365, 1999, 1037, 4418, 1012,  102]]
    - tokenTypeIds : [[0, 0, 0, 0, 0, 0, 0, 0, 0]]
    - mask : [[1, 1, 1, 1, 1, 1, 1, 1, 1]]
  - label : 4


In [8]:
dataset.validationExamples[0]

▿ 2 elements
  ▿ data : TextBatch
    - tokenIds : [[ 101, 1037, 2711, 3084, 1037, 2502, 3357, 2000, 1996, 2187, 1010, 2066, 2108, 3724, 1012,  102]]
    - tokenTypeIds : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
    - mask : [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
  - label : 0


## train

In [9]:
var optimizer = WeightDecayedAdam(
    for: bertClassifier,
    learningRate: LinearlyDecayedParameter(
        baseParameter: LinearlyWarmedUpParameter(
            baseParameter: FixedParameter<Float>(2e-5),
            warmUpStepCount: 10,
            warmUpOffset: 0),
        slope: -5e-7,  // The LR decays linearly to zero in 100 steps.
        startStep: 10),
    weightDecayRate: 0.01,
    maxGradientGlobalNorm: 1)

In [ ]:
print("Training BERT for the Language2Label task!")

for (epoch, epochBatches) in dataset.trainingEpochs.prefix(3).enumerated() {
    print("[Epoch \(epoch + 1)]")
    Context.local.learningPhase = .training
    var trainingLossSum: Float = 0
    var trainingBatchCount = 0
    print("epochBatches.count: \(epochBatches.count)")

    for batch in epochBatches {
        let (documents, labels) = (batch.data, Tensor<Int32>(batch.label))
        let (loss, gradients) = valueWithGradient(at: bertClassifier) { model -> Tensor<Float> in
            let logits = model(documents)
            return softmaxCrossEntropy(logits: logits, labels: labels)
        }

        trainingLossSum += loss.scalarized()
        trainingBatchCount += 1
        optimizer.update(&bertClassifier, along: gradients)

        print(
            """
              Training loss: \(trainingLossSum / Float(trainingBatchCount))
            """
        )
    }

    print("dataset.validationBatches.count: \(dataset.validationBatches.count)")
    Context.local.learningPhase = .inference
    var devLossSum: Float = 0
    var devBatchCount = 0
    var correctGuessCount = 0
    var totalGuessCount = 0

    for batch in dataset.validationBatches {
        let valBatchSize = batch.data.tokenIds.shape[0]

        let (documents, labels) = (batch.data, Tensor<Int32>(batch.label))
        let logits = bertClassifier(documents)
        let loss = softmaxCrossEntropy(logits: logits, labels: labels)
        devLossSum += loss.scalarized()
        devBatchCount += 1

        let correctPredictions = logits.argmax(squeezingAxis: 1) .== labels

        correctGuessCount += Int(Tensor<Int32>(correctPredictions).sum().scalarized())
        totalGuessCount += valBatchSize
    }
    
    let accuracy = Float(correctGuessCount) / Float(totalGuessCount)
    print(
        """
        Accuracy: \(correctGuessCount)/\(totalGuessCount) (\(accuracy)) \
        Eval loss: \(devLossSum / Float(devBatchCount))
        """
    )
}

Training BERT for the Language2Label task!
[Epoch 1]
epochBatches.count: 60
  Training loss: 1.4558853
  Training loss: 1.5414033
  Training loss: 1.564405
  Training loss: 1.5679815
  Training loss: 1.5503405


In [ ]:
struct Prediction {
    public let classIdx: Int
    public let className: String
    public let probability: Float
}

In [ ]:
// TODO: get num_best preds
func predict(_ texts: [String], bertClassifier: BERTClassifier) -> [Prediction] {
    print("predict()")
    print("texts: \(texts.count)")

    let validationExamples = texts.map {
        (text) -> TextBatch in
        return bertClassifier.bert.preprocess(
            sequences: [text],
            maxSequenceLength: maxSequenceLength
        )
    }
    
    print("validationExamples.count: \(validationExamples.count)")

    print("batchSize: \(batchSize)")
    print("maxSequenceLength: \(maxSequenceLength)")
    print("batchSize / maxSequenceLength: \(batchSize / maxSequenceLength)")

    let validationBatches = validationExamples.inBatches(of: batchSize / maxSequenceLength).map { 
        $0.paddedAndCollated(to: maxSequenceLength)
    }
    print("validationBatches: \(validationBatches.count)")
    var preds: [Prediction] = []
    for batch in validationBatches {
        print("batch")
        let logits = bertClassifier(batch)
        let probs = softmax(logits, alongAxis: 1)
        let classIdxs = logits.argmax(squeezingAxis: 1)
        let batchPreds = (0..<classIdxs.shape[0]).map { 
            (idx) -> Prediction in
            let classIdx: Int = Int(classIdxs[idx].scalar!)
            let prob = probs[idx, classIdx].scalar!
            return Prediction(classIdx: classIdx, className: dataset.labels[classIdx], probability: prob)
        }
        preds.append(contentsOf: batchPreds)
    }
    return preds
}

let texts = [
    "A person is walking forwards.", 
    "A person walks 4 steps forward.", 
    "A person walks in a circle counter clockwise.", 
    "A person getting done on their knees"
]

## do inference on whole dataset

In [ ]:
let dsURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/labels_ds_v2_manual_labels.csv")
let df = pd.read_csv(dsURL.path)

In [ ]:
let labels = df.label.unique().sorted().map {String($0)!}
labels

In [ ]:
let texts2: [String] = Array(df.text.to_list())! // .iloc[0..<2000]
texts2.count

In [ ]:
let preds2 = predict(texts2, bertClassifier: bertClassifier)

In [ ]:
// + save preds
// + new notebook
// + confusion matrix
// + review bad preds
// + review ambiguous preds
// + create manual labels
// * retrain bert
// - retrain img2label